In [1]:
import torch
#import nlp
from transformers import LongformerTokenizerFast

In [2]:
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096')

# Dataset processing

In [8]:
# with open("nq/train_wiki3_na_filtered_qg_t5l35-sqd_filtered.json", "r") as fh:
#     train_dataset = json.load(fh)

In [1]:
from datasets import load_dataset


dataset = load_dataset("natural_questions")

Using custom data configuration default
Reusing dataset natural_questions (/home/sri/.cache/huggingface/datasets/natural_questions/default/0.0.2/867dbbaf9137c1b83ecb19f5eb80559e1002ea26e702c6b919cfa81a17a8c531)


In [47]:
example = dataset['train'][1]

start_idx, end_idx = example['annotations']['long_answer'][0]['start_token'], example['annotations']['long_answer'][0]['end_token']

" ".join([itm for idx, itm in enumerate(example['document']['tokens']['token'][start_idx:end_idx]) if not example['document']['tokens']['is_html'][start_idx+idx]])

'Ctrl + Fn + F11 or Fn + B or Fn + Ctrl + B on certain Lenovo laptops and certain Dell laptops . Fn + Esc on Samsung . Ctrl + Fn + ⇧ Shift on certain HP laptops .'

In [21]:
import jsonlines

INPUT_FILE = "natural-questions/simplified-nq-train.jsonl"
START_TOKEN = 3521
END_TOKEN = 3525
QAS_ID = 4549465242785278785
REMOVE_HTML = True


def get_span_from_token_offsets(start_token, end_token, qas_id,
                                remove_html):
    for obj in f:
        if obj["example_id"] != qas_id:
            continue
        print("Found")
        return obj
        if remove_html:
            answer_span = [
                item["token"]
                for item in obj["document_tokens"][start_token:end_token]
                if not item["html_token"]
            ]
        else:
            answer_span = [
                item["token"]
                for item in obj["document_tokens"][start_token:end_token]
            ]

        return " ".join(answer_span)


with jsonlines.open(INPUT_FILE) as f:
    result = get_span_from_token_offsets(f, START_TOKEN, END_TOKEN, QAS_ID,
                                         REMOVE_HTML)

#print(result)

Found


In [246]:
for example in dataset['train']:
    if example['id']!='5648415780048191748':
        start_idx, end_idx = example['annotations']['long_answer'][0]['start_token'], example['annotations']['long_answer'][0]['end_token']
        answer  = " ".join([itm for idx, itm in enumerate(example['document']['tokens']['token'][start_idx:end_idx]) if not example['document']['tokens']['is_html'][start_idx+idx]])
        context = " ".join([itm for idx, itm in enumerate(example['document']['tokens']['token'][:]) if not example['document']['tokens']['is_html'][idx]])
        query = example['question']['text']
        input_pairs =[query, context]
        encodings = tokenizer.encode_plus(input_pairs, pad_to_max_length=True, max_length=4096)
        context_encodings = tokenizer.encode_plus(context)

        start_byte_idx = context.find(answer)
        end_byte_idx = start_byte_idx + len(answer)-1
        #print(start_idx)
        start_positions_context = context_encodings.char_to_token(start_byte_idx)
        end_positions_context = context_encodings.char_to_token(end_byte_idx)

        sep_idx = encodings['input_ids'].index(tokenizer.sep_token_id)
        print(len(encodings['input_ids']))
        #print(context,"\n\n" ,answer)
        start_positions = start_positions_context + sep_idx + 1
        end_positions = end_positions_context + sep_idx + 1

        if end_positions > 4096:
              start_positions, end_positions = 0, 0

                
        encodings.update({'start_positions': start_positions,
                              'end_positions': end_positions,
                              'attention_mask': encodings['attention_mask']})
        #break

4096
4096
4096
4096


OverflowError: can't convert negative int to unsigned

In [180]:
context[start_byte_idx:end_byte_idx]

"Over the next several months , pilots were trained , equipment was adapted , and intelligence was collected . Despite these preparations , Emperor Hirohito did not approve the attack plan until November 5 , after the third of four Imperial Conferences called to consider the matter . Final authorization was not given by the emperor until December 1 , after a majority of Japanese leaders advised him the `` Hull Note '' would `` destroy the fruits of the China incident , endanger Manchukuo and undermine Japanese control of Korea . ''"

In [181]:
end_byte_idx - start_byte_idx

536

In [247]:
def convert_to_features(example):
    try:
    #example = dataset['train'][0]
        start_idx, end_idx = example['annotations']['long_answer'][0]['start_token'], example['annotations']['long_answer'][0]['end_token']
        answer  = " ".join([itm for idx, itm in enumerate(example['document']['tokens']['token'][start_idx:end_idx]) if not example['document']['tokens']['is_html'][start_idx+idx]])
        context = " ".join([itm for idx, itm in enumerate(example['document']['tokens']['token'][:]) if not example['document']['tokens']['is_html'][idx]])
        query = example['question']['text']
        input_pairs =[query, context]
        encodings = tokenizer.encode_plus(input_pairs, pad_to_max_length=True, max_length=4096)
        context_encodings = tokenizer.encode_plus(context)

        start_byte_idx = context.find(answer)
        end_byte_idx = start_byte_idx + len(answer)-1

        if start_idx==-1:
            #print(start_idx)
            encodings.update({'start_positions': 0,
                              'end_positions': 0,
                              'attention_mask': encodings['attention_mask']})
            return encodings
        start_positions_context = context_encodings.char_to_token(start_byte_idx)
        end_positions_context = context_encodings.char_to_token(end_byte_idx)

        sep_idx = encodings['input_ids'].index(tokenizer.sep_token_id)

        start_positions = start_positions_context + sep_idx + 1
        end_positions = end_positions_context + sep_idx + 1

        if end_positions > 4096:
              start_positions, end_positions = 0, 0

        encodings.update({'start_positions': start_positions,
                              'end_positions': end_positions,
                              'attention_mask': encodings['attention_mask']})
    except Exception as e:
        #print(start_idx, example)
        print(example['id'])
    return encodings

In [203]:
train_dataset[8]['annotations']['long_answer']

[{'end_byte': 112155,
  'end_token': 2648,
  'start_byte': 111251,
  'start_token': 2553}]

In [248]:
train_dataset = dataset['train']
train_dataset = train_dataset.map(convert_to_features)

In [249]:
valid_dataset = dataset['validation']
valid_dataset = valid_dataset.map(convert_to_features, load_from_cache_file=False)

In [250]:
torch.save(train_dataset, 'data/train_data_full.pt')
torch.save(valid_dataset, 'data/valid_data_full.pt')

In [251]:
columns = ['input_ids', 'attention_mask', 'start_positions', 'end_positions']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)

In [252]:
len(train_dataset), len(valid_dataset)

(307373, 7830)

In [80]:
example['annotations']['long_answer'][0]

{'end_byte': 73287, 'end_token': 790, 'start_byte': 69526, 'start_token': 374}

In [253]:
# cach the dataset, so we can load it directly for training

torch.save(train_dataset, 'data/train_data.pt')
torch.save(valid_dataset, 'data/valid_data.pt')

In [3]:
# train_dataset = torch.load("data/train_data.pt")
# valid_dataset = torch.load("data/valid_data.pt")

# Training Script

In [4]:
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, List, Optional

import numpy as np
import torch

from transformers import LongformerForQuestionAnswering, LongformerTokenizerFast, EvalPrediction
from transformers import (
    HfArgumentParser,
    DataCollator,
    Trainer,
    TrainingArguments,
    set_seed,
)

logger = logging.getLogger(__name__)

In [5]:
@dataclass
class ModelArguments:
    """
    model/config/tokenizer arguments
    """
    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from hugginface.co/models"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )
    

@dataclass
class DataTrainingArguments:
    """
    model input arguments
    """
    train_file_path: Optional[str] = field(
        default='data/train_data.pt',
        metadata={"help": "Path for cached train dataset"}
    )
    valid_file_path: Optional[str] = field(
        default='data/valid_data.pt',
        metadata={"help": "Path for cached valid dataset"}
    )
    max_len: Optional[str] = field(
        default=4096,
        metadata={"help": "Max input length for the source text"}
    )
        

class DummyDataCollator():
    def __call__(self, batch: List) -> Dict[str, torch.Tensor]:
        """
        Take a list of samples from a Dataset and collate them into a batch.
        Returns:
            A dictionary of tensors
        """
        input_ids = torch.stack([example['input_ids'] for example in batch])
        attention_mask = torch.stack([example['attention_mask'] for example in batch])
        start_positions = torch.stack([example['start_positions'] for example in batch])
        end_positions = torch.stack([example['end_positions'] for example in batch])

        return {
            'input_ids': input_ids, 
            'start_positions': start_positions, 
            'end_positions': end_positions,
            'attention_mask': attention_mask
        }

In [6]:
def main():
    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
    
    model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath('args.json'))
    
    if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )
        
    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )
    logger.info("Training/evaluation parameters %s", training_args)

    # Set seed
    set_seed(training_args.seed)

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.
    
    tokenizer = LongformerTokenizerFast.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
    model = LongformerForQuestionAnswering.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
    
    # Get datasets
    print('loading data')
    train_dataset  = torch.load(data_args.train_file_path)
    valid_dataset = torch.load(data_args.valid_file_path)
    print('loading done')
    
    
    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=DummyDataCollator(),
 #       prediction_loss_only=True,
    )
    
    # Training
    if training_args.do_train:
        trainer.train(
            model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
        )
        trainer.save_model()
        # For convenience, we also re-save the tokenizer to the same directory,
        # so that you can share your model easily on huggingface.co/models =)
        if trainer.is_world_master():
            tokenizer.save_pretrained(training_args.output_dir)

    # Evaluation
    results = {}
    if training_args.do_eval and training_args.local_rank in [-1, 0]:
        logger.info("*** Evaluate ***")

        eval_output = trainer.evaluate()

        output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results *****")
            for key in sorted(eval_output.keys()):
                logger.info("  %s = %s", key, str(eval_output[key]))
                writer.write("%s = %s\n" % (key, str(eval_output[key])))
    
        results.update(eval_output)
    
    return results

def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()

In [14]:
import json

args_dict = {
  "n_gpu": 2,
  "model_name_or_path": 'allenai/longformer-base-4096',
  "max_len": 4096 ,
  "output_dir": './models',
  "overwrite_output_dir": True,
  "per_gpu_train_batch_size": 8,
  "per_gpu_eval_batch_size": 8,
  "gradient_accumulation_steps": 16,
  "learning_rate": 1e-4,
  "num_train_epochs": 3,
  "do_train": True
}

with open('args.json', 'w') as f:
    json.dump(args_dict, f)

In [15]:
main()

04/02/2021 10:11:45 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 2, distributed training: False, 16-bits training: False
04/02/2021 10:11:45 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=./models, overwrite_output_dir=True, do_train=True, do_eval=None, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=16, eval_accumulation_steps=None, learning_rate=0.0001, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Apr02_10-11-45_pop-os, logging_strategy=IntervalStrategy.STEPS, logging_first_step=False, logging_steps=500, save_strategy=IntervalStrategy.STEPS, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level=O1, f

loading data
loading done


Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/sri/miniconda3/envs/dl/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/sri/miniconda3/envs/dl/lib/python3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/home/sri/miniconda3/envs/dl/lib/python3.8/site-packages/transformers/models/longformer/modeling_longformer.py", line 1988, in forward
    outputs = self.longformer(
  File "/home/sri/miniconda3/envs/dl/lib/python3.8/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/home/sri/miniconda3/envs/dl/lib/python3.8/site-packages/transformers/models/longformer/modeling_longformer.py", line 1668, in forward
    encoder_outputs = self.encoder(
  File "/home/sri/miniconda3/envs/dl/lib/python3.8/site-pa

In [236]:
for d in  train_dataset:
    print (d['input_ids'].shape)

torch.Size([6423])
torch.Size([4096])
torch.Size([4096])
torch.Size([8298])
torch.Size([4096])
torch.Size([4096])
torch.Size([7672])
torch.Size([7985])
torch.Size([21720])
torch.Size([4096])
torch.Size([4096])
torch.Size([37835])
torch.Size([16348])
torch.Size([4096])
torch.Size([4096])
torch.Size([5981])
torch.Size([5932])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([6693])
torch.Size([5397])
torch.Size([5967])
torch.Size([8351])
torch.Size([17922])
torch.Size([31421])
torch.Size([6096])
torch.Size([17593])
torch.Size([7101])
torch.Size([4558])
torch.Size([12074])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4316])
torch.Size([4096])
torch.Size([19739])
torch.Size([12279])
torch.Size([14599])
torch.Size([5210])
torch.Size([23179])
torch.Size([4096])
torch.Size([4259])
torch.Size([5160])
torch.Size([4096])
torch.Size([4096])
torch.Size([5504])
torch.Size([4096])
torch.Size([4096])
torch.Size([34150])
torch.Size([31218])
torch.Size([9512])

KeyboardInterrupt: 

In [237]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'question', 'annotations'],
        num_rows: 307373
    })
    validation: Dataset({
        features: ['id', 'document', 'question', 'annotations'],
        num_rows: 7830
    })
})

# args testing

In [7]:
from transformers import (
HfArgumentParser,
TrainingArguments,
)

import argparse


def get_train_args():
    parser = argparse.ArgumentParser('Train spanBert model on NQ')

    parser.add_argument('--name',
                        type=str,
                        default='SpanBert',
                        help='Train Name which we use to identify the training. Need not be unique.')
    parser.add_argument('--train_file_path',
                        type=str,
                        default='data/train_data.pt',
                        help='Path for cached train dataset')
    parser.add_argument('--valid_file_path',
                        type=str,
                        default='data/valid_data.pt',
                        help='Path for cached valid dataset')
    parser.add_argument('--max_len',
                        type=str,
                        default=4096,
                        help='Max input length for the source text.')
    parser.add_argument('--num_epochs',
                        type=int,
                        default=3,
                        help='Number of epochs for which to train. Negative means forever.')
    parser.add_argument('--n_gpu',
                        type=int,
                        default=1,
                        help='Number of GPU to train the model on.')
    parser.add_argument('--save_dir',
                        type=str,
                        default='./save',
                        help='Directory to store the model in (If already exists, use --overwrite_output_dir).')
    parser.add_argument('--overwrite_output_dir',
                        type=bool,
                        default=True,
                        help='Overwrite existing model directory')
    parser.add_argument('--per_device_train_batch_size',
                        type=int,
                        default=8)
    parser.add_argument('--per_gpu_eval_batch_size',
                        type=int,
                        default=8)
    parser.add_argument('--gradient_accumulation_steps',
                        type=int,
                        default=16)
    parser.add_argument('--learning_rate',
                        type=float,
                        default=1e-4,
                        help='Learning rate for the model')
    parser.add_argument('--num_train_epochs',
                        type=int,
                        default=3,
                        help='Number of epochs for which to train. Negative means forever.')
    parser.add_argument('--do_train',
                        type=bool,
                        default=True,
                        help='Whether to train the model.')
    parser.add_argument('--seed',
                        type=int,
                        default=42,
                        help='Seed to reproduce.')
    parser.add_argument('--do_eval',
                        type=bool,
                        default=True,
                        help='Whether to evaluate the model after training.')
    parser.add_argument('--local_rank',
                        type=int,
                        default=-1)

    args = parser.parse_args()
    return args

In [4]:
TrainingArguments

transformers.training_args.TrainingArguments

In [1]:
from args import get_train_args
parser = get_train_args()

usage: Train spanBert model on NQ [-h]
                                  [--model_name_or_path MODEL_NAME_OR_PATH]
                                  [--tokenizer_name TOKENIZER_NAME]
                                  [--cache_dir CACHE_DIR] [--name NAME]
                                  [--train_file_path TRAIN_FILE_PATH]
                                  [--valid_file_path VALID_FILE_PATH]
                                  [--max_len MAX_LEN]
                                  [--num_epochs NUM_EPOCHS] [--n_gpu N_GPU]
                                  [--save_dir SAVE_DIR]
                                  [--overwrite_output_dir OVERWRITE_OUTPUT_DIR]
                                  [--per_device_train_batch_size PER_DEVICE_TRAIN_BATCH_SIZE]
                                  [--per_gpu_eval_batch_size PER_GPU_EVAL_BATCH_SIZE]
                                  [--gradient_accumulation_steps GRADIENT_ACCUMULATION_STEPS]
                                  [--learning_rate LEARNING_RATE]

SystemExit: 2

/home/sri/miniconda3/envs/dl/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
